In [4]:
import pandas as pd
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import metrics
from sklearn.metrics import accuracy_score

import string

from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
df_dataset = pd.read_csv('./output/US_count_vectorizer_dataset.csv')
df_dataset.head()

,title,filter_title
0,WE WANT TO TALK ABOUT OUR MARRIAGE,we want to talk about our marriage
1,The Trump Presidency: Last Week Tonight with J...,the trump presidency last week tonight with jo...
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",racist superman rudy mancuso king bach lele pons
3,Nickelback Lyrics: Real or Fake?,nickelback lyrics real or fake
4,I Dare You: GOING BALD!?,i dare you going bald


In [3]:
df_x = df_dataset['filter_title']
df_x2 = df_dataset['filter_title_no_stops']
df_y = df_dataset['category_id']
target_names = list(df_dataset['category_id'].unique())

KeyError: 'filter_title_no_stops'

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=117)
x_train2, x_test2 = train_test_split(df_x2, test_size=0.2, random_state=117)

# replace NaN with spaces so count vectorizer can work
x_train2 = x_train2.fillna(' ')
x_test2 = x_test2.fillna(' ')



In [ ]:
count_vectorizer = CountVectorizer()
train_count_vector = count_vectorizer.fit_transform(x_train)
test_count_vector = count_vectorizer.transform(x_test)
print(train_count_vector.shape)
# print(count_vectorizer.vocabulary_)
print(train_count_vector.toarray())

train_count_vector2 = count_vectorizer.fit_transform(x_train2)
test_count_vector2 = count_vectorizer.transform(x_test2)


In [ ]:
tfidf_vectorizer = TfidfTransformer()
x_trained_tfidf_vector = tfidf_vectorizer.fit_transform(train_count_vector)
x_test_tfidf_vector = tfidf_vectorizer.transform(test_count_vector)
print(x_trained_tfidf_vector.shape)
# print(count_vectorizer.vocabulary_)
print(x_trained_tfidf_vector.toarray())

In [ ]:
LogReg = LogisticRegression(solver='lbfgs', max_iter = 4000)
normalizedLogRegFit = LogReg.fit(x_trained_tfidf_vector, y_train)
normalized_predict = normalizedLogRegFit.predict(x_test_tfidf_vector)
print("Accuracy: ", accuracy_score(y_test, normalized_predict))
print(metrics.classification_report(y_test, normalized_predict, target_names=target_names))

In [ ]:
RegularLogRegFit = LogReg.fit(train_count_vector, y_train)
regular_predict = RegularLogRegFit.predict(test_count_vector)
print("Accuracy: ", accuracy_score(y_test, regular_predict))
print(metrics.classification_report(y_test, regular_predict, target_names=target_names))

In [ ]:
stopWordsLogRegFit = LogReg.fit(train_count_vector2, y_train)
stopWords_predict = RegularLogRegFit.predict(test_count_vector2)
print("Accuracy: ", accuracy_score(y_test, stopWords_predict))
print(metrics.classification_report(y_test, stopWords_predict, target_names=target_names))